In [1]:
import sys
sys.path.append('/Users/dongyuli/isi/repos/datamart')
from datamart import search, augment
from datamart.utilities.utils import Utils
import pandas as pd
import json

In [6]:
old_df = pd.read_csv("/Users/dongyuli/isi/repos/datamart/example/fifa_example/fifa.csv")
print("- READ THE SUPPLY DATASET -\n")
print(old_df.head())

- READ THE SUPPLY DATASET -

   d3mIndex  GameID        Date          Team      Opponent  \
0         0       0  14-06-2018        Russia  Saudi Arabia   
1         1       1  14-06-2018  Saudi Arabia        Russia   
2         2       2  15-06-2018         Egypt       Uruguay   
3         3       3  15-06-2018       Uruguay         Egypt   
4         4       4  15-06-2018       Morocco          Iran   

   Ball Possession %  Off-Target  Blocked  Offsides  Saves  Pass Accuracy %  \
0                 40           3        3         3      0               78   
1                 60           3        3         1      2               86   
2                 43           3        2         1      3               78   
3                 57           6        4         1      3               86   
4                 64           6        4         0      2               86   

   Passes  Distance Covered (Kms)  Yellow & Red  Man of the Match  1st Goal  \
0     306                     118     

In [3]:
with open("/Users/dongyuli/isi/repos/datamart/example/fifa_example/fifa_query.json") as f_json:
    query_json = json.load(f_json)
results = search(query_json, old_df)
print("- SEARCH DATAMART BY A DESCRIPTION JSON OBJECT -\n")
print("Returned %d Datasets" % len(results))
for res in results:
    print(res.id)
    print('\t' + '\n\t'.join(json.dumps(res.metadata, indent=2).split('\n', 50)[:50]))
    print('\t... ...')

- SEARCH DATAMART BY A DESCRIPTION JSON OBJECT -

Returned 1 Datasets
127860000
	{
	  "datamart_id": 127860000,
	  "title": "FIFA World Cup",
	  "description": "FIFA World Cup",
	  "url": "https://www.football-data.org",
	  "keywords": [
	    "football",
	    "competition"
	  ],
	  "provenance": {
	    "source": "www.football-data.org"
	  },
	  "materialization": {
	    "python_path": "football_match_materializer",
	    "arguments": {
	      "uri": "/v2/competitions/2000/matches?limit=999",
	      "token": "d019bc4541c9490fabcba6806cbcc42b"
	    }
	  },
	  "variables": [
	    {
	      "datamart_id": 127860001,
	      "name": "id",
	      "semantic_type": [
	        "http://schema.org/Integer"
	      ],
	      "description": "column name: id, dtype: int64"
	    },
	    {
	      "datamart_id": 127860002,
	      "name": "season_id",
	      "semantic_type": [
	        "http://schema.org/Integer"
	      ],
	      "description": "column name: season_id, dtype: int64"
	    },
	    {
	      "d

In [7]:
new_df = Utils.get_dataset(metadata=results[0].metadata)
print("- MATERIALIZE SEARCH RESULTS -\n")
print(new_df.head())

- MATERIALIZE SEARCH RESULTS -

       id  season_id season_startDate season_endDate  season_currentMatchday  \
0  200000          1       2018-06-14     2018-07-15                       3   
1  200001          1       2018-06-14     2018-07-15                       3   
2  200006          1       2018-06-14     2018-07-15                       3   
3  200007          1       2018-06-14     2018-07-15                       3   
4  200012          1       2018-06-14     2018-07-15                       3   

                utcDate    status  matchday        stage    group  \
0  2018-06-14T15:00:00Z  FINISHED       1.0  GROUP_STAGE  Group A   
1  2018-06-15T12:00:00Z  FINISHED       1.0  GROUP_STAGE  Group A   
2  2018-06-15T15:00:00Z  FINISHED       1.0  GROUP_STAGE  Group B   
3  2018-06-15T18:00:00Z  FINISHED       1.0  GROUP_STAGE  Group B   
4  2018-06-16T10:00:00Z  FINISHED       1.0  GROUP_STAGE  Group C   

            ...           referees_5_nationality referees_6_id  \
0     

In [8]:
target_dataset = results[0]

print("- AUGMENT BY THE SEARCHED RESULT -\n")
result = augment(old_df, target_dataset)
print(result.head())

- AUGMENT BY THE SEARCHED RESULT -

   d3mIndex  GameID        Date          Team      Opponent  \
0         0       0  14-06-2018        Russia  Saudi Arabia   
1         1       1  14-06-2018  Saudi Arabia        Russia   
2         2       2  15-06-2018         Egypt       Uruguay   
3         3       3  15-06-2018       Uruguay         Egypt   
4         4       4  15-06-2018       Morocco          Iran   

   Ball Possession %  Off-Target  Blocked  Offsides  Saves  \
0                 40           3        3         3      0   
1                 60           3        3         1      2   
2                 43           3        2         1      3   
3                 57           6        4         1      3   
4                 64           6        4         0      2   

            ...            referees_5_nationality  referees_6_id  \
0           ...                              None          43878   
1           ...                              None          49527   
2       